In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

In [2]:
import os
os.chdir ('/home/ubuntu/kaggle/liberty/')
os.getcwd()

'/home/ubuntu/kaggle/liberty'

In [3]:
#load train and test
train =pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [4]:
train.head()

,Hazard,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,T1_V9,...,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
Id,,,,,,,,,,,,,,,,,,,,,
1,1,15,3,2,N,B,N,B,B,D,...,2,37,1,11,6,Y,N,E,2,2
2,4,16,14,5,H,B,N,B,B,C,...,2,22,1,18,5,Y,Y,E,2,1
3,1,10,10,5,N,K,N,B,B,E,...,6,37,2,14,6,Y,Y,E,6,1
4,1,18,18,5,N,K,N,B,B,E,...,2,25,1,1,6,Y,N,C,2,6
5,1,13,19,5,N,H,N,B,B,E,...,1,22,1,2,7,N,N,E,1,1


In [5]:
train.shape

(50999, 33)

In [6]:
test.shape

(51000, 32)

In [7]:
labels = train.Hazard
train.drop('Hazard', axis=1, inplace=True)

In [8]:
columns= train.columns
test_ind = test.index

In [9]:
type(columns)

pandas.core.index.Index

In [10]:
train = np.array(train)
test = np.array(test)

In [11]:
# label encode the categorical variables
for i in range(train.shape[1]):
    if type(train[1,i]) is str:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[:,i]) + list(test[:,i]))
        train[:,i] = lbl.transform(train[:,i])
        test[:,i] = lbl.transform(test[:,i])

In [12]:
train = train.astype(float)
test = test.astype(float)

In [42]:
params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.001
params["min_child_weight"] = 5
params["subsample"] = 0.8
params["scale_pos_weight"] = 1.0
params["colsample_bytree"] = 0.725
params["scale_pos_weight"] = 1.0
params["silent"] = 1
params["max_depth"] = 7

In [43]:
plst = list(params.items())

In [47]:
plst

[('colsample_bytree', 0.725),
 ('subsample', 0.8),
 ('min_child_weight', 5),
 ('silent', 1),
 ('eta', 0.01),
 ('objective', 'reg:linear'),
 ('scale_pos_weight', 1.0),
 ('max_depth', 7)]

In [16]:
type(plst)

list

In [17]:
#Using 5000 rows for early stopping
offset = 5000

In [46]:
num_rounds = 2500

In [19]:
xgtest = xgb.DMatrix(test)

In [20]:
xgtest

In [21]:
#create a train and validation matrices
xgtrain = xgb.DMatrix(train[offset:,:], label = labels[offset:])
xgval = xgb.DMatrix(train[:offset,:], label =labels[:offset])

In [48]:
#train using early stopping and predict
watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=5)
preds1 = model.predict(xgtest)

Will train until val error hasn't decreased in 5 rounds.
[0]	train-rmse:5.326006	val-rmse:5.269031
[1]	train-rmse:5.300390	val-rmse:5.243824
[2]	train-rmse:5.275544	val-rmse:5.219334
[3]	train-rmse:5.250580	val-rmse:5.194692
[4]	train-rmse:5.226002	val-rmse:5.170086
[5]	train-rmse:5.201534	val-rmse:5.146332
[6]	train-rmse:5.178363	val-rmse:5.123370
[7]	train-rmse:5.154725	val-rmse:5.100259
[8]	train-rmse:5.131482	val-rmse:5.077512
[9]	train-rmse:5.108526	val-rmse:5.055100
[10]	train-rmse:5.085958	val-rmse:5.033005
[11]	train-rmse:5.063967	val-rmse:5.011674
[12]	train-rmse:5.042036	val-rmse:4.990688
[13]	train-rmse:5.020751	val-rmse:4.969716
[14]	train-rmse:4.999581	val-rmse:4.948932
[15]	train-rmse:4.978716	val-rmse:4.928694
[16]	train-rmse:4.958063	val-rmse:4.908604
[17]	train-rmse:4.938010	val-rmse:4.889237
[18]	train-rmse:4.918091	val-rmse:4.869855
[19]	train-rmse:4.898528	val-rmse:4.850798
[20]	train-rmse:4.879674	val-rmse:4.832695
[21]	train-rmse:4.860847	val-rmse:4.814255
[22]	tr

In [49]:
train = train[::-1,:]
labels =labels[::-1]

In [50]:
xgbtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
xgval  = xgb.DMatrix(train[:offset,:], label=labels[:offset])

In [51]:
watchlist = [(xgtrain, 'train'), (xgval, 'val')]
model= xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=5)
preds2= model.predict(xgtest)

Will train until val error hasn't decreased in 5 rounds.
[0]	train-rmse:5.326006	val-rmse:5.230930
[1]	train-rmse:5.300390	val-rmse:5.204907
[2]	train-rmse:5.275544	val-rmse:5.179603
[3]	train-rmse:5.250872	val-rmse:5.154511
[4]	train-rmse:5.226243	val-rmse:5.129646
[5]	train-rmse:5.202452	val-rmse:5.105425
[6]	train-rmse:5.178595	val-rmse:5.081410
[7]	train-rmse:5.154910	val-rmse:5.057355
[8]	train-rmse:5.131487	val-rmse:5.033822
[9]	train-rmse:5.108560	val-rmse:5.010566
[10]	train-rmse:5.085936	val-rmse:4.987857
[11]	train-rmse:5.063689	val-rmse:4.965457
[12]	train-rmse:5.041900	val-rmse:4.943351
[13]	train-rmse:5.021098	val-rmse:4.922121
[14]	train-rmse:5.000033	val-rmse:4.900721
[15]	train-rmse:4.979131	val-rmse:4.879706
[16]	train-rmse:4.958436	val-rmse:4.859015
[17]	train-rmse:4.938888	val-rmse:4.839261
[18]	train-rmse:4.919640	val-rmse:4.819565
[19]	train-rmse:4.900569	val-rmse:4.799734
[20]	train-rmse:4.881284	val-rmse:4.780428
[21]	train-rmse:4.862547	val-rmse:4.761247
[22]	tr

In [52]:
preds = preds1 + preds2

In [53]:
type(preds)

numpy.ndarray

In [54]:
preds = pd.DataFrame({"Id": test_ind, "Hazard": preds})
preds = preds.set_index('Id')
preds.to_csv('xgboost_bm_july7th_5.csv')